In [32]:
import os
from docx import Document
import pandas as pd
import re
# openai предоставляет только библиотеку для формирования запросов к языковым моделям
# ChatGPT не используется
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Dict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


def load_documents_from_folder(path, prefix):
    # Собираем список файлов в папке
    files = [f for f in os.listdir(path) if f.endswith('.docx') and f.startswith(prefix)]
    data = []
    for file in files:
        # Извлекаем номер из названия файла
        match = re.search(rf'{prefix}(\d+)', file)
        if match:
            file_number = match.group(1)
        else:
            continue  # Пропускаем файлы без номера

        # Полный путь к файлу
        filepath = os.path.join(path, file)
        doc = Document(filepath)
        paragraphs = [p.text for p in doc.paragraphs]  # Извлекаем текст из всех абзацев
        if paragraphs:
            first_line = paragraphs[0]
            name_match = re.search(r'\](.*)', first_line)
            if name_match:
                document_name = name_match.group(1).strip()  # Извлекаем название документа
            else:
                document_name = "No Name Found"  # Заглушка, если название не найдено
        else:
            document_name = "Empty Document"  # Заглушка для пустых документов

        full_text = '\n'.join(paragraphs)  # Объединяем текст в одну строку
        data.append({'Number': file_number, 'Name': document_name, 'Text': full_text})
    return pd.DataFrame(data)

# Пути к папкам
path_hmi = './HMI'
path_stss = './SSTS'

# Загружаем документы из обеих папок
df_hmi = load_documents_from_folder(path_hmi, 'UC-')
df_stss = load_documents_from_folder(path_stss, 'SSTS-')

# Объединяем датафреймы по номеру документа
df = pd.merge(df_hmi, df_stss, on='Number', how='left', suffixes=('_HMI', '_SSTS'))

# Дропаем колонку Name_SSTS
df = df.drop(columns=['Name_SSTS'])

# Переименовываем колонку Name_HMI в Name
df = df.rename(columns={'Name_HMI': 'Name'})

print(df)


    Number                                           Name  \
0    30365     Start the Charging Process via Soft Switch   
1   259572              Mute/unmute the FM Radio playback   
2    29448                    Configure heat preservation   
3    65833             Driver initiate a call through SWP   
4   261611                    FM Radio Stations switching   
5   315231        Adding Internet Radio to Favorites List   
6    30364  Stop charging when the cut-off SOC is reached   
7    65832             Driver initiate a call through SWP   
8    86921  Emergency Service Communication (ERA-Glonass)   
9    30370         Stop the discharging process (Updated)   
10   88001                   Receiving Call Notifications   
11   88002                   Receiving Call Notifications   
12  259571              Mute/unmute the FM Radio playback   
13   65831             Driver initiate a call through SWP   

                                             Text_HMI  \
0   [I-30365]  Start the Ch

In [33]:
from typing import List, Optional
import pandas as pd
import requests
from pydantic import BaseModel

class Message(BaseModel):
    role: str
    content: str
    
# Payload example
class Payload(BaseModel):
    model: Optional[str] = None
    messages: List[Message]

# Функция `sent_to_ai` предназначена для отправки запроса в ЛОКАЛЬНО развернутую llm модель
# Данные никуда не отправляются и остаются в безопасности на вашем устройстве
# Она принимает данные в формате словаря (payload), которые содержат параметры для выполнения запроса к модели
# В результате выполнения запроса функция получает ответ от модели и возвращает его текстовое содержание

# DEFAULT_MODEL = 'llama-3.2-3b-instruct'
DEFAULT_MODEL = 'meta-llama-3.1-8b-instruct'
    
def sent_to_ai(payload: Payload):
    if not payload.model:
        payload.model = DEFAULT_MODEL
        
    payload = payload.dict()
    
    try:
        client = OpenAI(base_url="http://127.0.0.1:1234/v1/", api_key="lm-studio")
        response = client.chat.completions.create(**payload)
        print('response', response)
        
        return response.choices[0].message.content
    except Exception as e:
        # Обработка любой ошибки
        print(f"Произошла ошибка: {e}")




In [34]:
def create_short_hmi_description(text_hmi):
    prompt = f"""
    Analyze the following detailed description of an HMI (Human-Machine Interface) and summarize it in a concise and clear format. Focus on the main functions, user interactions, and essential controls or indicators. The summary should be brief, straightforward, and formatted in a way that makes it easy for further verification or testing.

    Original HMI Description: "{text_hmi}"

    Please provide a short summary focusing on the key functionalities and interactions in the HMI. Avoid any additional explanations, and keep the language simple and direct.
    """
    messages = [Message(role="user", content=prompt)]
    payload = Payload(messages=messages)
    response = sent_to_ai(payload)
    return response.strip()  # Возвращаем краткое описание без лишних символов

# Применение функции для создания новой колонки 'Short HMI'
df['Short HMI'] = df['Text_HMI'].apply(create_short_hmi_description)

response ChatCompletion(id='chatcmpl-wkfigawz72z8f3ousrynq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**HMI Summary: Start Charging Process**\n\n* **User Interaction:** \n  - Navigate to charging interface\n  - Press "Start Charging" button\n* **Preconditions:**\n  - Vehicle in Parking mode\n  - Charging cable plugged in\n* **System Actions:**\n  - Check vehicle and cable status\n  - Send start charging command\n  - Display notification of successful start\n* **Indicators:** \n  - Charging process is in progress\n\n**Controls:**\n\n- Soft switch button: "Start Charging"\n- Notifications: display on SWP or Mobile App', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1731224089, model='meta-llama-3.1-8b-instruct', object='chat.completion', service_tier=None, system_fingerprint='meta-llama-3.1-8b-instruct', usage=CompletionUsage(completion_tokens=117, prompt_tokens=360, total_tokens=477

In [35]:
# Шаблон промпта для фильтрации нерелевантного контента
keep_only_relevant_content_prompt_template = """
You are provided with a query and a set of retrieved documents. Your task is to filter out all non-relevant information that does not provide important details regarding the query. 

Query: {query}

Retrieved Documents: {retrieved_documents}

Your goal is to keep only the information that is directly relevant to the query, removing any extraneous details. You may remove parts of sentences or entire sentences that are not relevant to the query.

DO NOT ADD ANY NEW INFORMATION THAT IS NOT IN THE RETRIEVED DOCUMENTS.

Output only the filtered, relevant content.
"""

# Модель для вывода релевантного контента
class KeepRelevantContent(BaseModel):
    relevant_content: str = Field(description="The relevant content from the retrieved documents that is relevant to the query.")

# Подготовка промпта для запроса
def create_relevant_content_prompt(short_hmi, text_stss):
    return keep_only_relevant_content_prompt_template.format(
        query=short_hmi,
        retrieved_documents=text_stss
    )

# Функция для фильтрации контента
def keep_only_relevant_content(row):
    """
    Filters Text_SSTS to keep only the content relevant to Short HMI.

    Args:
        row: A row of the DataFrame containing 'Short HMI' and 'Text_SSTS'.

    Returns:
        str: The filtered relevant content from 'Text_SSTS' based on 'Short HMI'.
    """
    # Проверяем, есть ли данные в Text_STSS
    if pd.isna(row['Text_SSTS']):
        return ""

    # Формируем промпт для фильтрации контента
    prompt = create_relevant_content_prompt(row['Short HMI'], row['Text_SSTS'])
    messages = [Message(role="user", content=prompt)]
    payload = Payload(messages=messages)
    response = sent_to_ai(payload)

    # Возвращаем отфильтрованный релевантный контент
    return response.strip()

# Применение функции к DataFrame для обновления колонки 'Text_STSS'
df['Filtered Text_SSTS'] = df.apply(keep_only_relevant_content, axis=1)

response ChatCompletion(id='chatcmpl-gghm97oemmi1j3px90a31m', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Filtered Relevant Content**\n\n**Preconditions:**\n- Vehicle in Parking mode\n- Charging cable plugged in\n\n**User Interaction:**\n- Navigate to charging interface\n- Press "Start Charging" button\n\n**System Actions:**\n- Check vehicle and cable status\n- Send start charging command\n- Display notification of successful start\n\n**Indicators:**\n- Charging process is in progress\n\n**Execution output (related to starting charging):**\na. IVI_IFT sends vehicle/{VIN}/hardware/battery/chargingAppointCommand=0x1:Charge ON to SGW, SGW sends IVI_AppointChargeCmd=0x1:Charge ON to VCU\n\nNote: The above execution output corresponds to the "immediate charging" switch in the original document, which is related to starting the charging process.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], cre

In [36]:
def answer_question_from_context(row):
    """
    Answers a question about whether 'Short HMI' meets the requirements in 'Text_SSTS' using a chain-of-thought reasoning approach.

    Args:
        row: A row of the DataFrame containing 'Short HMI' and 'Text_SSTS'.

    Returns:
        str: The answer based on the reasoning chain.
    """
    # Формируем контекст и вопрос для цепочки рассуждений
    context = f"""
    Short HMI: {row['Short HMI']}
    Requirements (Text_SSTS): {row['Text_SSTS']}

    """
    
    question = "Does the 'Short HMI' meet the requirements outlined in 'Text_SSTS'?"

    # Промпт с примерами цепочки рассуждений
    prompt = f"""
    Examples of Chain-of-Thought Reasoning

    Example 1
    Context:
    Short HMI: "The vehicle's dashboard displays the current speed in kilometers per hour."
    Requirements (Text_SSTS): "The vehicle must display the speed in both kilometers per hour and miles per hour."
    Question: Does the 'Short HMI' meet the requirements outlined in 'Text_SSTS'?
    Reasoning Chain:
    The Short HMI states that the dashboard displays the current speed in kilometers per hour.
    The requirement specifies that the speed must be displayed in both kilometers per hour and miles per hour.
    Since the Short HMI only mentions kilometers per hour, it does not include miles per hour.
    Therefore, the 'Short HMI' does not fully meet the requirements outlined in 'Text_SSTS'.

    Example 2
    Context:
    Short HMI: "The vehicle is equipped with a touchscreen interface that allows users to control the air conditioning system, including temperature settings, fan speed, and airflow direction."
    Requirements (Text_SSTS): "The human-machine interface must provide controls for the air conditioning system, including temperature adjustment, fan speed control, and airflow direction selection."
    Question: Does the 'Short HMI' meet the requirements outlined in 'Text_SSTS'?
    Reasoning Chain:
    The Short HMI describes a touchscreen interface that allows users to control temperature settings, fan speed, and airflow direction.
    The requirements specify that the HMI must provide controls for temperature adjustment, fan speed control, and airflow direction selection.
    The Short HMI includes all the controls specified in the requirements.
    Therefore, the 'Short HMI' meets the requirements outlined in 'Text_SSTS'.

    Now, use the following context and question to provide an answer with step-by-step reasoning:
    Context:
    {context}
    Question:
    {question}
    """

    # Подготовка сообщений и отправка запроса к GPT
    messages = [Message(role="user", content=prompt)]
    payload = Payload(messages=messages)
    response = sent_to_ai(payload)
    # Возвращаем только ответ
    return response.strip()

# Применение функции к DataFrame для создания колонки с ответами
df['Requirement Compliance Answer'] = df.apply(answer_question_from_context, axis=1)



response ChatCompletion(id='chatcmpl-ozni1g7dmno0c3jud2muh', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='To determine if the \'Short HMI\' meets the requirements outlined in \'Text_SSTS\', we will analyze each step of the chain-of-thought reasoning.\n\nThe Short HMI: **HMI Summary: Start Charging Process** includes user interactions, preconditions, system actions, and indicators. We need to compare this with the requirements outlined in \'Text_SSTS\'.\n\n1.  The first requirement states that enabling conditions (a&b&c) must be met:\n    *   a. The power mode of the vehicle is CGW_VehicleState=Standby-Normal/ StandbyOnlyLVon/Drive-Normal;\n    *   b. The user can operate the display;\n    *   c. IVI_IFT can receive the status signal of the corresponding controller;\n\nThe Short HMI does not explicitly mention these enabling conditions or provide any information about the power mode, user interaction with the display, or the abilit

In [38]:
def summarize_discrepancies(description):
    prompt = f"""
    Given a detailed description of compliance discrepancies, summarize the key issues succinctly. Focus only on the main points of non-compliance and essential details that highlight what is wrong or missing. Please provide a concise summary.

    Full Description: "{description}"
    """
    messages = [Message(role="user", content=prompt)]
    payload = Payload(messages=messages)  # Используйте соответствующую модель GPT
    response = sent_to_ai(payload)
    return response.strip()

# Применение функции к DataFrame для создания новой колонки 'Differences'
df['Differences'] = df['Requirement Compliance Answer'].apply(summarize_discrepancies)


response ChatCompletion(id='chatcmpl-mri9csyra3fpgnne3k3dji', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Here's a concise summary of the key issues:\n\n**Non-Compliance Summary:**\n\n1.  **Enabling Conditions**: The Short HMI does not mention or satisfy the enabling conditions, including:\n    *   Power mode (CGW_VehicleState)\n    *   User interaction with the display\n    *   IVI_IFT's ability to receive status signals from the controller\n2.  **Trigger Conditions**: The Short HMI lacks specific trigger conditions and corresponding statuses that IVI_IFT must receive, including:\n    *   Charging states (AC/DC Preheating, AC/DC Charging)\n    *   Charging pile and connector states\n    *   Charging lock state\n3.  **Execution Output**: The Short HMI does not display necessary switches for stopping charging or unlocking/electronic locking based on specific conditions.\n\nThese key issues highlight the main points of non-complian

In [39]:
def create_description(row):
    prompt = f"""
    Given the detailed descriptions from three sources - HMI interface operations, system standards and specifications (STSS), and key discrepancies - synthesize a clear and concise summary that captures the essence of the functionality or process described. Focus on integrating information to provide a comprehensive yet succinct description that is useful for understanding the core functionalities and any critical issues.

    HMI Description: "{row['Text_HMI']}"
    STSS Description: "{row['Text_SSTS']}"
    Key Discrepancies: "{row['Differences']}"

    Please produce a summary that combines these details into a streamlined description of the main features and functions, highlighting any significant discrepancies where applicable. The summary should be concise, informative, and only include the most relevant information for clarity and utility.
    """
    messages = [Message(role="user", content=prompt)]
    payload = Payload(messages=messages)
    response = sent_to_ai(payload)
    return response.strip()

# Применение функции к DataFrame
df['Description'] = df.apply(create_description, axis=1)


response ChatCompletion(id='chatcmpl-7gu40obu3ea5g50oicz3', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Charging Process Overview:**\n\nThe charging process can be initiated through various interfaces, including the Soft Switch on the vehicle\'s display (HMI), mobile app (STSS), or IVI_IFT interface. The system checks if the vehicle is in Parking mode and the charging cable is plugged in before starting the charging process.\n\n**Main Features:**\n\n1.  **Trigger Conditions:** The charging process can be started when the following conditions are met:\n    *   Vehicle is in Parking mode\n    *   Charging cable is plugged in\n    *   Specific charging states (AC/DC Preheating, AC/DC Charging) or statuses (e.g., charging connector state, lock state) are received by IVI_IFT.\n2.  **Execution Output:** Once the charging process starts:\n    *   The system displays a notification that the charging session has been successfully starte

In [40]:
# Модель данных для проверки фактической обоснованности ответа
class IsGroundedOnFacts(BaseModel):
    grounded_on_facts: str = Field(description="Answer 'True' if the answer is based on the provided context, 'False' otherwise.")

# Шаблон промпта для проверки фактической обоснованности ответа
is_grounded_on_facts_prompt_template = """
You are provided with a context and an answer. Your task is to determine if the answer is fully grounded in the information provided within the context. 

Context:
{context}

Answer:
{answer}

Please respond with "True" if the answer is completely based on the given context, without any unsupported assumptions or additional information, or "False" if the answer includes any hallucinations or unsupported claims.
"""

# Функция для создания промпта
def create_is_grounded_prompt(context, answer):
    return is_grounded_on_facts_prompt_template.format(
        context=context,
        answer=answer
    )

# Основная функция для проверки обоснованности ответа
def is_answer_grounded_on_context(state):
    """
    Determines if the answer to the question is grounded in the facts.
    
    Args:
        state: A dictionary containing the context and answer.
    
    Returns:
        str: "grounded on context" if the answer is based on the context, "hallucination" otherwise.
    """
    print("Checking if the answer is grounded in the facts...")
    context = state["context"]
    answer = state["answer"]

    # Создание промпта для проверки
    prompt = create_is_grounded_prompt(context, answer)
    messages = [Message(role="user", content=prompt)]
    payload = Payload(messages=messages)
    response = sent_to_ai(payload)

    # Проверка ответа: True - обоснован, False - содержит галлюцинации
    if response.strip().lower() == "true":
        print("The answer is grounded in the facts.")
        return "grounded on context"
    else:
        print("The answer is hallucination.")
        return "hallucination"


In [41]:
def send_to_gpt_for_rating(row):
    # Проверка на применимость требований
    if pd.isna(row['Text_SSTS']):
        return "NA"  # Возвращаем "NA", если требования не применимы

    prompt = f"""
    Given comprehensive descriptions from multiple sources, analyze the text and classify it into one of the specific compliance categories based solely on the content provided. You should return ONLY the abbreviation of the compliance level without any additional text or explanation.

    Categories:
    - FC (Fully Compliant): The situation is perfect and nothing can be improved.
    - LC (Largely Compliant): Generally correct, but some minor improvements may be needed. No full review is necessary.
    - PC (Partially Compliant): There are major deviations. Significant improvements are needed and a subsequent review is required.
    - NC (Non-Compliant): The requirements are not met, necessitating a complete redo and re-review.
    - NA (Not Applicable): The situation described does not apply to the standards or requirements in question.

    HMI Description: "{row['Text_HMI']}"
    System Standards and Specifications (SSTS): "{row['Text_SSTS']}"
    Key Discrepancies: "{row['Requirement Compliance Answer']}"
    Differences: "{row['Differences']}"
    Integrated Description: "{row['Description']}"

    Based on the above information, please provide only the two-letter abbreviation (FC, LC, PC, NC, NA) that best describes the overall compliance level.
    """
    messages = [Message(role="user", content=prompt)]
    payload = Payload(messages=messages)
    response = sent_to_ai(payload)
    return response.strip()  # Ensure only the abbreviation is returned with no extra spaces or characters

# Применение функции к DataFrame
df['Complience Level'] = df.apply(send_to_gpt_for_rating, axis=1)



response ChatCompletion(id='chatcmpl-k3xic9uroa1amf5jai1bz', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='PC', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1731224328, model='meta-llama-3.1-8b-instruct', object='chat.completion', service_tier=None, system_fingerprint='meta-llama-3.1-8b-instruct', usage=CompletionUsage(completion_tokens=1, prompt_tokens=3579, total_tokens=3580, completion_tokens_details=None, prompt_tokens_details=None))
response ChatCompletion(id='chatcmpl-yfs8pusg07oqvip3p4r8w', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='PC', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1731224329, model='meta-llama-3.1-8b-instruct', object='chat.completion', service_tier=None, system_fingerprint='meta-llama-3.1-8b-instruct', usage=CompletionUsage(completion_tokens=1, prompt_token

In [42]:
submission = df[['Number', 'Name', 'Differences', 'Description', 'Complience Level']]
submission.to_csv('submission.csv', index = False)
print(submission)

    Number                                           Name  \
0    30365     Start the Charging Process via Soft Switch   
1   259572              Mute/unmute the FM Radio playback   
2    29448                    Configure heat preservation   
3    65833             Driver initiate a call through SWP   
4   261611                    FM Radio Stations switching   
5   315231        Adding Internet Radio to Favorites List   
6    30364  Stop charging when the cut-off SOC is reached   
7    65832             Driver initiate a call through SWP   
8    86921  Emergency Service Communication (ERA-Glonass)   
9    30370         Stop the discharging process (Updated)   
10   88001                   Receiving Call Notifications   
11   88002                   Receiving Call Notifications   
12  259571              Mute/unmute the FM Radio playback   
13   65831             Driver initiate a call through SWP   

                                          Differences  \
0   Here's a concise summar